In [1]:
import numpy as np
import pandas as pd

### <a id = "p2a">2a.</a>  <font color = "green"> Download data from Bloomberg</font>  [back to table of contents](#top)

In [2]:
etf_prices_df = pd.read_excel("ETFs.xlsx",
                            sheet_name="Prices",
                            index_col=0,
                            skiprows=[0,1,2,3,4,6])

# # Drop first row if there is requesting data issue
# etf_prices_df = etf_prices_df[1:]

# Convert index into datetime
etf_prices_df.index = pd.to_datetime(etf_prices_df.index)

# Clean up the ticker names
etf_prices_df.columns = etf_prices_df.columns.str.split().str[0]

etf_prices_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
2000-01-03,14.6454,NaN,13.7393,11.2453,18.2495,41.4953,12.4788,NaN,11.3398,20.7667,...,NaN,NaN,NaN,9.90,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,14.5044,NaN,13.4806,10.7537,17.7454,39.3901,12.1274,NaN,10.9978,20.2955,...,NaN,NaN,NaN,9.68,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,15.1036,NaN,13.8362,10.6692,17.6663,38.8053,12.3417,NaN,11.2753,20.1134,...,NaN,NaN,NaN,9.67,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,15.4913,NaN,14.3697,11.1378,17.9035,37.5188,12.5731,NaN,11.2559,20.1777,...,NaN,NaN,NaN,9.66,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,15.4825,NaN,14.5232,11.3221,18.5857,38.1738,13.3701,NaN,11.3528,20.4240,...,NaN,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,83.2300,97.45,80.5500,49.4500,134.1500,215.8100,81.4800,41.56,80.7500,135.3600,...,96.66,88.92,192.00,46.88,197.42,276.11,118.58,131.58,384.67,167.61
2025-05-08,84.3400,97.68,81.6100,49.8600,136.0000,217.8700,81.3800,41.36,80.1000,134.1400,...,96.42,88.57,192.72,47.00,201.18,278.28,118.97,132.16,387.98,168.31
2025-05-09,84.5600,97.18,82.4300,49.8600,136.1100,217.6000,80.9100,41.61,80.2400,132.6800,...,97.07,89.16,192.23,47.00,200.81,277.97,119.13,132.47,387.74,168.03
2025-05-12,86.5200,99.20,84.5800,50.8700,140.3000,227.7000,81.0300,41.65,79.7600,135.9600,...,98.12,89.60,197.08,47.88,207.87,287.27,122.03,134.91,403.93,171.54


This section takes way too long, that is why summary was created

In [3]:
def read_columns_from_excel(file_path, column_name):
  """
  Reads a specific column from every sheet in an Excel file and combines them into a single DataFrame.

  Args:
    file_path (str): The path to the Excel file.
    column_name (str): The name of the column to read from each sheet.

  Returns:
    pandas.DataFrame: A DataFrame containing the specified column from each sheet,
                      with an added column indicating the sheet name.
                      Returns an empty DataFrame if an error occurs or the file is empty.
  """
  try:
    excel_file = pd.ExcelFile(file_path)
    all_sheets = {}

    for sheet_name in excel_file.sheet_names:
      df = pd.read_excel(file_path,
                            sheet_name=sheet_name,
                            index_col=0,
                            skiprows=[0,1,2,3,4,5])
      if column_name in df.columns:
        all_sheets[sheet_name] = df[column_name]
      else:
        print(f"Column '{column_name}' not found in sheet '{sheet_name}'. Skipping.")

    if not all_sheets:
      return pd.DataFrame()

    combined_df = pd.concat(all_sheets, axis=1)
    combined_df.columns.name = None
    combined_df = combined_df.reset_index().rename(columns={'index': 'row_index'})
    return combined_df

  except Exception as e:
      print(f"An error occurred: {e}")
      return pd.DataFrame()

In [4]:
market_cap = read_columns_from_excel("ETFs.xlsx", "CUR_MKT_CAP")
market_cap.ffill(axis=0, inplace=True)
market_cap.rename(columns = {"row_index":"Date"},inplace=True)
market_cap.set_index("Date",inplace=True,drop=True)
market_cap 

Column 'CUR_MKT_CAP' not found in sheet 'Prices'. Skipping.
Column 'CUR_MKT_CAP' not found in sheet 'Current'. Skipping.
Column 'CUR_MKT_CAP' not found in sheet 'Notes'. Skipping.


,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,116.8596,NaN,223.1250,347.7000,129.7971,1117.0660,270.7250,NaN,119.4210,130.2767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,118.3065,NaN,218.9250,341.2500,126.2111,1113.0190,270.8401,NaN,115.8187,133.2423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,135.2456,NaN,228.7125,346.1645,125.6486,1096.4960,275.6250,NaN,118.7415,132.0471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,138.7174,NaN,237.5301,361.3680,127.3361,1045.1060,280.7933,NaN,118.5375,132.4688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,138.6382,NaN,240.0677,367.3492,135.1250,1081.2000,296.1563,NaN,120.9314,135.5759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,4820.9916,20810.4475,26156.5343,49227.2489,19448.5304,66503.1241,15910.7470,7393.5240,18509.8509,35324.2663,...,1842.0126,33277.1965,86794.2866,NaN,58732.4500,448829.2954,20514.3400,19480.9003,152790.0589,130030.8112
2025-05-08,4956.9760,20923.0560,26402.8100,49560.6120,19709.9360,67105.2448,15992.9448,7368.2840,18485.0102,35495.4996,...,1837.4390,33146.3078,87129.9485,NaN,60696.0060,452382.8888,20581.8100,19560.1634,153930.9719,130624.6476
2025-05-09,4948.7662,20684.7630,26701.0718,49757.5590,19678.2394,67043.8432,15823.7158,7412.8216,18561.4501,34571.8053,...,1849.8258,33367.7715,86889.3408,NaN,60584.3770,451978.7005,20609.4895,19599.4213,153827.6396,130374.3408


### <a id = "p2b">2b.</a>  <font color = "green"> Download data from Bloomberg</font>  [back to table of contents](#top)

In [5]:
etf_prices_df = pd.read_excel("ETF_Summary.xlsx",
                            sheet_name="Prices",
                            index_col=0,
                            skiprows=[0,1,2,3,4,6])

# # Drop first row if there is requesting data issue
# etf_prices_df = etf_prices_df[1:]

# Convert index into datetime
etf_prices_df.index = pd.to_datetime(etf_prices_df.index)

# Clean up the ticker names
etf_prices_df.columns = etf_prices_df.columns.str.split().str[0]

etf_prices_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
2000-01-03,14.6454,NaN,13.7393,11.2453,18.2495,41.4953,12.4788,NaN,11.3398,20.7667,...,NaN,NaN,NaN,9.90,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,14.5044,NaN,13.4806,10.7537,17.7454,39.3901,12.1274,NaN,10.9978,20.2955,...,NaN,NaN,NaN,9.68,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,15.1036,NaN,13.8362,10.6692,17.6663,38.8053,12.3417,NaN,11.2753,20.1134,...,NaN,NaN,NaN,9.67,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,15.4913,NaN,14.3697,11.1378,17.9035,37.5188,12.5731,NaN,11.2559,20.1777,...,NaN,NaN,NaN,9.66,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,15.4825,NaN,14.5232,11.3221,18.5857,38.1738,13.3701,NaN,11.3528,20.4240,...,NaN,NaN,NaN,9.83,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,83.2300,97.45,80.5500,49.4500,134.1500,215.8100,81.4800,41.56,80.7500,135.3600,...,96.66,88.92,192.00,46.88,197.42,276.11,118.58,131.58,384.67,167.61
2025-05-08,84.3400,97.68,81.6100,49.8600,136.0000,217.8700,81.3800,41.36,80.1000,134.1400,...,96.42,88.57,192.72,47.00,201.18,278.28,118.97,132.16,387.98,168.31
2025-05-09,84.5600,97.18,82.4300,49.8600,136.1100,217.6000,80.9100,41.61,80.2400,132.6800,...,97.07,89.16,192.23,47.00,200.81,277.97,119.13,132.47,387.74,168.03
2025-05-12,86.5200,99.20,84.5800,50.8700,140.3000,227.7000,81.0300,41.65,79.7600,135.9600,...,98.12,89.60,197.08,47.88,207.87,287.27,122.03,134.91,403.93,171.54


In [6]:
market_cap_df = pd.read_excel("ETF_Summary.xlsx",
                            sheet_name="MARKET_CAP",
                            index_col=0,
                            skiprows=[0,1,2,3,4,6])

# # Drop first row if there is requesting data issue
# market_cap_df = market_cap_df[1:]

# Convert index into datetime
market_cap_df.index = pd.to_datetime(market_cap_df.index)

# Clean up the ticker names
market_cap_df.columns = market_cap_df.columns.str.split().str[0]

market_cap_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
2000-01-03,116.8596,NaN,223.1250,347.7000,129.7971,1117.0660,270.7250,NaN,119.4210,130.2767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,118.3065,NaN,218.9250,341.2500,126.2111,1113.0190,270.8401,NaN,115.8187,133.2423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,135.2456,NaN,228.7125,346.1645,125.6486,1096.4960,275.6250,NaN,118.7415,132.0471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,138.7174,NaN,237.5301,361.3680,127.3361,1045.1060,280.7933,NaN,118.5375,132.4688,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,138.6382,NaN,240.0677,367.3492,135.1250,1081.2000,296.1563,NaN,120.9314,135.5759,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,4820.9916,20810.4475,26156.5343,49227.2489,19448.5304,66503.1241,15910.7470,7393.5240,18509.8509,35324.2663,...,1842.0126,33277.1965,86794.2866,NaN,58732.4500,448829.2954,20514.3400,19480.9003,152790.0589,130030.8112
2025-05-08,4956.9760,20923.0560,26402.8100,49560.6120,19709.9360,67105.2448,15992.9448,7368.2840,18485.0102,35495.4996,...,1837.4390,33146.3078,87129.9485,NaN,60696.0060,452382.8888,20581.8100,19560.1634,153930.9719,130624.6476
2025-05-09,4948.7662,20684.7630,26701.0718,49757.5590,19678.2394,67043.8432,15823.7158,7412.8216,18561.4501,34571.8053,...,1849.8258,33367.7715,86889.3408,NaN,60584.3770,451978.7005,20609.4895,19599.4213,153827.6396,130374.3408
2025-05-12,5059.1467,21174.2400,27410.1958,50928.2668,20291.0278,70224.0227,15867.4409,7384.5453,18486.3075,35711.9792,...,1916.4423,33550.6107,89037.6885,NaN,62828.7075,467202.2077,21111.1898,19953.6830,160292.5622,133180.3931


In [7]:
put_call_df = pd.read_excel("ETF_Summary.xlsx",
                            sheet_name="PUT_CALL",
                            index_col=0,
                            skiprows=[0,1,2,3,4,6])

# # Drop first row if there is requesting data issue
# put_call_df = put_call_df[1:]

# Convert index into datetime
put_call_df.index = pd.to_datetime(put_call_df.index)

# Clean up the ticker names
put_call_df.columns = put_call_df.columns.str.split().str[0]

put_call_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,1.2304,1.6963,1.6877,1.7763,2.3103,1.5490,2.3621,2.6548,2.5588,1.4437,...,9.8571,1.4333,0.1815,NaN,2.1267,0.6314,1.4998,0.3815,1.1695,0.3641
2025-05-08,1.2244,1.7349,1.6844,1.7836,2.3077,1.5463,2.3147,2.6180,2.6316,1.4442,...,9.8571,1.4264,0.1823,NaN,2.1052,0.6250,1.5005,0.3796,1.1690,0.3572
2025-05-09,1.2041,1.7353,1.6699,1.8000,1.8375,1.5424,2.2926,2.6335,2.6044,1.4448,...,9.4138,1.4568,0.1823,NaN,2.0912,0.6212,1.5003,0.3856,1.1600,0.3528
2025-05-12,1.2206,1.6513,1.6702,1.8009,1.8713,1.5588,2.3098,2.6092,2.5928,1.4619,...,9.4138,1.4551,0.1822,NaN,2.0475,0.6191,1.4966,0.3866,1.1559,0.3500


In [8]:
short_interest_df = pd.read_excel("ETF_Summary.xlsx",
                            sheet_name="SHORT_INT",
                            index_col=0,
                            skiprows=[0,1,2,3,4,6])

# # Drop first row if there is requesting data issue
# short_interest_df = short_interest_df[1:]

# Convert index into datetime
short_interest_df.index = pd.to_datetime(short_interest_df.index)

# Clean up the ticker names
short_interest_df.columns = short_interest_df.columns.str.split().str[0]

short_interest_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,...,RWR,VNQ,VIG,VBINX,IWM,VTI,ACWI,SDY,VUG,VTV
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### <a id = "p2c">2c.</a>  <font color = "green"> Get macroeconomic data</font>  [back to table of contents](#top)

In [9]:
Actual_release = pd.read_excel("US_Economic_Data.xlsx",
                            sheet_name="Actual",
                            index_col=0,
                            skiprows=[0,1,2,3,5,6])

# # Drop first row if there is requesting data issue
# Actual_release = Actual_release[1:]

# Convert index into datetime
Actual_release.index = pd.to_datetime(Actual_release.index)

# # Clean up the ticker names
# Actual_release.columns = Actual_release.columns.str.split().str[0]

Actual_release

,Change in Non-farm payrolls,Change in private payrolls,Unemployment Rate,AHE MoM,AHE YoY,Initial Jobless Claims,Continuing Claims,ADP Employment,CPI MoM,Core CPI MoM,...,Durable Goods Orders,US Trade Balance,US Empire Manufacturing,Retail Sales Advance MoM,US Industrial Production MoM,Building Permits,Housing Starts,Existing Home Sales,Fed Funds Target - Upper,Fed Funds Target - Lower
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
Latest_release = pd.read_excel("US_Economic_Data.xlsx",
                            sheet_name="Latest",
                            index_col=0,
                            skiprows=[0,1,2,3,5,6])

# # Drop first row if there is requesting data issue
# Latest_release = Latest_release[1:]

# Convert index into datetime
Latest_release.index = pd.to_datetime(Latest_release.index)

# # Clean up the ticker names
# Latest_release.columns = Latest_release.columns.str.split().str[0]

Latest_release

,Change in Non-farm payrolls,Change in private payrolls,Unemployment Rate,AHE MoM,AHE YoY,Initial Jobless Claims,Continuing Claims,ADP Employment,CPI MoM,Core CPI MoM,...,US Trade Balance,US Empire Manufacturing,Retail Sales Advance MoM,US Industrial Production MoM,Building Permits,Housing Starts,Existing Home Sales,Fed Funds Target - Upper,Fed Funds Target - Lower,US Federal Funds Effective Rate
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.00,14.00
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.00,13.89
1980-01-04,NaN,NaN,NaN,NaN,NaN,394.0,2749.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.00,14.00
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.00,14.00
1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.00,13.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.25,4.33
2025-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.25,4.33
2025-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.25,4.33
2025-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.25,NaN


In [11]:
Release_date = pd.read_excel("US_Economic_Data.xlsx",
                            sheet_name="ReleaseDate",
                            index_col=0,
                            skiprows=[0,1,2,3,5,6])

# # Drop first row if there is requesting data issue
# Release_date = Release_date[1:]

# Convert index into datetime
Release_date.index = pd.to_datetime(Release_date.index)

# # Clean up the ticker names
# Release_date.columns = Release_date.columns.str.split().str[0]

Release_date

,Change in Non-farm payrolls,Change in private payrolls,Unemployment Rate,AHE MoM,AHE YoY,Initial Jobless Claims,Continuing Claims,ADP Employment,CPI MoM,Core CPI MoM,...,Durable Goods Orders,US Trade Balance,US Empire Manufacturing,Retail Sales Advance MoM,US Industrial Production MoM,Building Permits,Housing Starts,Existing Home Sales,Fed Funds Target - Upper,Fed Funds Target - Lower
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-02,NaN,NaN,NaN,NaN,NaN,20250508.0,20250515.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def convert_float_columns_to_datetime(df):
    """
    Converts all columns with float values in a Pandas DataFrame to datetime objects.

    Args:
        df (pd.DataFrame): The input DataFrame.
        unit (str, optional): The unit of the float values (e.g., 's' for seconds, 'ms' for milliseconds). Defaults to 's'.

    Returns:
        pd.DataFrame: The DataFrame with float columns converted to datetime.
    """
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = pd.to_datetime(df[col], format='%Y%m%d')
    return df

In [13]:
Release_date = convert_float_columns_to_datetime(Release_date)
Release_date

,Change in Non-farm payrolls,Change in private payrolls,Unemployment Rate,AHE MoM,AHE YoY,Initial Jobless Claims,Continuing Claims,ADP Employment,CPI MoM,Core CPI MoM,...,Durable Goods Orders,US Trade Balance,US Empire Manufacturing,Retail Sales Advance MoM,US Industrial Production MoM,Building Permits,Housing Starts,Existing Home Sales,Fed Funds Target - Upper,Fed Funds Target - Lower
1980-01-02,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1980-01-03,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1980-01-04,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1980-01-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1980-01-08,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2025-05-02,NaT,NaT,NaT,NaT,NaT,2025-05-08,2025-05-15,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2025-05-05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2025-05-06,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [14]:
# Create the new DataFrame with the same structure
Adjusted_Actual_release = pd.DataFrame(index=Actual_release.index,
                                       columns=Actual_release.columns)

# Iterate through rows and columns
for row in range(len(Actual_release)):
    for column in range(Actual_release.shape[1]):
        value = Actual_release.iloc[row, column]
        if pd.notnull(value) and np.issubdtype(type(value), np.number):
            column_name = Actual_release.columns[column]
            Data_point = value
            Actual_release_date = Release_date.iloc[row, column]
            Adjusted_Actual_release.loc[Actual_release_date, column_name] = Data_point

Adjusted_Actual_release.sort_index(axis=0, 
                                   level=None, 
                                   ascending=True, 
                                   inplace=True)

Adjusted_Actual_release = Adjusted_Actual_release[Adjusted_Actual_release.index.notnull()]

Adjusted_Actual_release.ffill(axis=0, inplace=True)

# Display result
Adjusted_Actual_release

/var/folders/9f/g6tpsc8n5qb861y9f7rpwlcm0000gn/T/ipykernel_48950/118669104.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Adjusted_Actual_release.ffill(axis=0, inplace=True)


,Change in Non-farm payrolls,Change in private payrolls,Unemployment Rate,AHE MoM,AHE YoY,Initial Jobless Claims,Continuing Claims,ADP Employment,CPI MoM,Core CPI MoM,...,Durable Goods Orders,US Trade Balance,US Empire Manufacturing,Retail Sales Advance MoM,US Industrial Production MoM,Building Permits,Housing Starts,Existing Home Sales,Fed Funds Target - Upper,Fed Funds Target - Lower
1980-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-01,228.0,209.0,4.2,0.3,3.8,241.0,1841.0,62.0,-0.1,0.1,...,1.0,-122.662,-8.1,1.4,-0.3,1467.0,1324.0,4.02,4.5,4.25
2025-05-02,177.0,167.0,4.2,0.2,3.8,241.0,1841.0,62.0,-0.1,0.1,...,9.2,-122.662,-8.1,1.4,-0.3,1467.0,1324.0,4.02,4.5,4.25
2025-05-05,177.0,167.0,4.2,0.2,3.8,241.0,1841.0,62.0,-0.1,0.1,...,9.2,-122.662,-8.1,1.4,-0.3,1467.0,1324.0,4.02,4.5,4.25
2025-05-06,177.0,167.0,4.2,0.2,3.8,241.0,1841.0,62.0,-0.1,0.1,...,9.2,-140.498,-8.1,1.4,-0.3,1467.0,1324.0,4.02,4.5,4.25
